In [40]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

### Read and clean csv

In [41]:
df = pd.read_csv("las5.csv").drop(columns=["Timestamp", "Empty"]).rename(columns={
    "What is your name?": "name",
    "How many office hours are you doing?": "hours",
    "For each of the following, choose which days and times you are available (Please try to put at LEAST 10 available slots)": "times"
}).dropna()
df.head()

,name,hours,times
0,Sarah Bost (210),5,"Monday 10-11, Tues 11-12, Tues 12-1, Tues 1-2,..."
1,Shannon Goad (401),4,"Monday 2-3, Tues 1-2, Wed 1-2, Wed 2-3, Wed 3-..."
2,Dana Rubin (401),4,"Monday 10-11, Monday 1-2, Monday 4-5, Tues 10-..."
3,Ruchi Sarkar (210),4,"Monday 1-2, Tues 10-11, Tues 11-12, Wed 10-11,..."
4,Nidhi Murlidhar (401),4,"Tues 10-11, Tues 11-12, Tues 12-1, Tues 1-2, W..."


### Manually Modify Times

In [42]:
from typing import List

def modifyTimes(name: str, la_class: int, times_to_add: List[str], times_to_remove: List[str]) -> bool:
    
    full_name = str(name + " (" + str(la_class) + ")")
    
    if full_name not in str(df["name"]):
        print(full_name + " not present")
        return False
    
    if len(times_to_add) > 0:
        for t in times_to_add:
            if t.split(" ")[0] not in ["Monday", "Tues", "Wed", "Thur"] or \
               t.split(" ")[1] not in ["10-11", "11-12","12-1", "1-2", "2-3", "3-4", "4-5"]:
                print(str(t) + " not in correct format")
                return False
        
    if len(times_to_remove) > 0:
        for t in times_to_remove:
            if t.split(" ")[0] not in ["Monday", "Tues", "Wed", "Thur"] or \
               t.split(" ")[1] not in ["10-11", "11-12","12-1", "1-2", "2-3", "3-4", "4-5"]:
                print(str(t) + " not in correct format")
                return False
    # to do
    #   make sure name/class combination is present
    #   make sure time to remove is present
    #   make sure time to add and time to remove are in the correct form
    
    for index, row in df.iterrows():
        if row["name"] == full_name:
            
            times = df.at[index, 'times'].split(", ")
            original_times = times
            
            if len(times_to_add) > 0:
                for t in times_to_add:
                    if t in times:
                        print(full_name + " already has time " + str(t))
                    else:
                        times.append(t)
                        print(t + " has been added to " + name)
            
            if len(times_to_remove) > 0:
                for t in times_to_remove:
                    if t not in times:
                        print(full_name + " does not have time " + str(t) + " to remove")
                    else:
                        times.remove(t)
                        print(t + " has been removed from " + name)
            
            df.at[index, 'times'] = str(", ".join(times))
            
    print(full_name + " now has times: " + str(times))
    
    return True

In [43]:
# PLACE FUNCTION CALLS BELOW:

# EXAMPLE: modifyTimes("James Bury", 401, ["Monday 4-5"], ["Tuesday 1-2"])
# modifyTimes("Anna Truelove", 401, ["Wed 10-11"], ["Thur 1-2"])
# modifyTimes("Sarah Bost", 210, ["Wed 10-11", "Thur 10-11"], ["Monday 3-4", "Monday 4-5"])

### Sort times

In [44]:
all_times = set()

for index, row in df.iterrows():
    times = str(row["times"]).split(", ")
    for t in times:
        if t not in all_times:
            all_times.add(t)
            
all_times = list(all_times)

In [45]:
days = ["Monday", "Tues", "Wed", "Thur"]
time_order = ["10-11", "11-12", "12-1", "1-2", "2-3", "3-4", "4-5"]
sorted_times = []

for d in days:
    curr_day = [x for x in all_times if d in x]
    for t in time_order:
        for c in curr_day:
            if t in c:
                sorted_times.append(c)

### Create LA and Shift classes

In [46]:
class LA:
    
    def __init__(self, name, hours):
        self.name = name
        self.hours_left = hours + 1
        self.hours_committed = hours + 1
        self.shifts = []
        self.possible_shifts = []
        
    def assign(self, shift):
        if self.hours_left > 0 and \
           (shift not in self.shifts) and \
           self.consecutivePreviousShifts(shift) < 4:
            self.hours_left -= 1
            self.shifts.append(shift)
            return True
        else:
            return False
        
    def consecutivePreviousShifts(self, shift):
        if shift.previous_shift_title == "" or shift.previous_shift_title not in [x.title for x in self.shifts]:
            return 0
        else:
            prev = list(filter(lambda x: x.title == shift.previous_shift_title, self.shifts))
            return 1 + self.consecutivePreviousShifts(prev[0])
        
    def addPossibleShift(self, shift):
        self.possible_shifts.append(shift)
        
    def removePossibleShift(self, shift):
        self.possible_shifts.remove(shift)
        
    def __repr__(self):
        return "name: " + self.name + ", hours left: " + str(self.hours_left) + ", shifts: " + str([x.title for x in self.shifts])
        
        
class Shift:
    
    def __init__(self, title):
        self.title = title
        self.workers = []
        self.available_workers = []
        self.positions_remaining = 6
        self.previous_shift_title = ""
        
        if sorted_times.index(self.title) > 0 and \
           sorted_times[sorted_times.index(self.title) - 1].split(" ")[0] == self.title.split(" ")[0]:
            self.previous_shift_title = sorted_times[sorted_times.index(self.title) - 1]
        
    def addLAToShift(self, la):
        if self.positions_remaining > 0 and (la not in self.workers):
            if la.assign(self):
                self.workers.append(la)
                self.positions_remaining -= 1
                return True
        else:
            return False
        
    def addAvailableWorker(self, la):
        self.available_workers.append(la)
        
    def removeAvailableWorker(self, la):
        self.available_workers.remove(la)
        
    def __repr__(self):
        return "Shift: " + self.title + ", LAs assigned: " + str([x.name for x in self.workers]) + \
               ", positions left: " + str(self.positions_remaining) + ", available las: " + \
                str([x.name for x in self.available_workers])

### Add time columns, create lists of LAs and shifts

In [47]:
LAs = []
shifts = []

for t in sorted_times:
    df[t] = 0
    shifts.append(Shift(t))
    
for index, row in df.iterrows():
    l = LA(row["name"], row["hours"])
    for t in sorted_times:
        if t in str(row["times"]).split(", "):
            df[t][index] = 1
            l.addPossibleShift(t)
            [x for x in shifts if x.title == t][0].addAvailableWorker(l)
    
    LAs.append(l)

### Sorts shift times based on LAs available

In [48]:
workers = {}

for t in sorted_times:
    workers[t] = sum(df[t])

# Sort by fewest LAs available
sorted_workers = {k: v for k, v in sorted(workers.items(), key=lambda item: item[1])}
print(sorted_workers)
print(sorted_times)

{'Tues 3-4': 1, 'Wed 11-12': 2, 'Thur 3-4': 2, 'Monday 11-12': 3, 'Tues 2-3': 3, 'Wed 12-1': 3, 'Thur 4-5': 3, 'Monday 12-1': 4, 'Monday 3-4': 4, 'Monday 4-5': 4, 'Tues 10-11': 4, 'Thur 10-11': 4, 'Thur 2-3': 4, 'Tues 12-1': 5, 'Tues 4-5': 5, 'Wed 3-4': 5, 'Wed 4-5': 5, 'Thur 1-2': 5, 'Monday 1-2': 6, 'Monday 2-3': 6, 'Tues 11-12': 6, 'Wed 2-3': 6, 'Thur 11-12': 6, 'Thur 12-1': 6, 'Monday 10-11': 7, 'Tues 1-2': 7, 'Wed 10-11': 7, 'Wed 1-2': 8}
['Monday 10-11', 'Monday 11-12', 'Monday 12-1', 'Monday 1-2', 'Monday 2-3', 'Monday 3-4', 'Monday 4-5', 'Tues 10-11', 'Tues 11-12', 'Tues 12-1', 'Tues 1-2', 'Tues 2-3', 'Tues 3-4', 'Tues 4-5', 'Wed 10-11', 'Wed 11-12', 'Wed 12-1', 'Wed 1-2', 'Wed 2-3', 'Wed 3-4', 'Wed 4-5', 'Thur 10-11', 'Thur 11-12', 'Thur 12-1', 'Thur 1-2', 'Thur 2-3', 'Thur 3-4', 'Thur 4-5']


### Assigns shifts

In [49]:
runs_since_changed = 0

while not runs_since_changed >= 29:
    for t in sorted_workers.keys():
        current_shift = [x for x in shifts if x.title == t][0]
        for l in current_shift.available_workers:
            if current_shift.addLAToShift(l):
                runs_since_changed = 0
                break
    runs_since_changed += 1

### Manually Add Shifts After Scheduling

In [50]:
def ManualChange(la_name: str, shift_title: str, add: bool) -> bool:
    
    if len([x for x in LAs if x.name == la_name]) == 0:
        print(la_name + " is not an LA. You may have forgotten the class")
        return False
    if len([x for x in shifts if x.title == shift_title]) == 0:
        print(shift_title + " is not a shift.")
        return False
    
    la_to_assign = [x for x in LAs if x.name == la_name][0]
    shift_to_assign = [x for x in shifts if x.title == shift_title][0]
    full_name = la_to_assign.name
    
    # Make sure name is valid
    if full_name not in str(df["name"]):
        print(full_name + " not present")
        return False
    
    # Make sure time is valid
    t = shift_to_assign.title
    if t.split(" ")[0] not in ["Monday", "Tues", "Wed", "Thur"] or \
       t.split(" ")[1] not in ["10-11", "11-12","12-1", "1-2", "2-3", "3-4", "4-5"]:
        print(str(t) + " not in correct format")
        return False
    
    # Make sure LA isn't already working that shift
    if (add and la_to_assign.name in [x.name for x in shift_to_assign.workers]) or \
        ((not add) and la_to_assign.name not in [x.name for x in shift_to_assign.workers]):
        print(la_to_assign.name + " is already listed/not listed in " + shift_to_assign.title)
        return False
    
    if (add and shift_to_assign.title in [x.title for x in la_to_assign.shifts]) or \
        ((not add) and shift_to_assign.title not in [x.title for x in la_to_assign.shifts]):
        print(shift_to_assign.title + " is already listed/not listed in " + la_to_assign.name + "\'s times")
        return False
    
    if add:
        la_to_assign.shifts.append(shift_to_assign)
        la_to_assign.hours_left -= 1
        shift_to_assign.workers.append(la_to_assign)
        shift_to_assign.positions_remaining -= 1
        print(la_to_assign)
        print(shift_to_assign)

        if la_to_assign.hours_left < 0:
            print("WARNING: " + la_to_assign.name + " is working too many hours")

        if shift_to_assign.positions_remaining < 0:
            print("WARNING: " + shift_to_assign.title + " has too many LAs")
    else:
        la_to_assign.shifts.remove(shift_to_assign)
        la_to_assign.hours_left += 1
        shift_to_assign.workers.remove(la_to_assign)
        shift_to_assign.positions_remaining += 1
        print(la_to_assign)
        print(shift_to_assign)
    
    return True

In [51]:
# ManualChange("Caleb Kang (401)", "Monday 4-5", False)
# ManualChange("Caleb Kang (401)", "Wed 3-4", True)

# Anna
ManualChange(la_name="Anna Truelove (401)", shift_title="Wed 11-12", add=True)

# Amy
ManualChange(la_name="Amy Clark (401)", shift_title="Thur 1-2", add=True)

# Shannon
ManualChange(la_name="Shannon Goad (401)", shift_title="Tues 1-2", add=False)
ManualChange(la_name="Shannon Goad (401)", shift_title="Wed 1-2", add=True)

name: Anna Truelove (401), hours left: 0, shifts: ['Thur 3-4', 'Monday 11-12', 'Tues 2-3', 'Thur 2-3', 'Monday 10-11', 'Wed 11-12']
Shift: Wed 11-12, LAs assigned: ['Halie Chmura (401)', 'Akash Krishna (210)', 'Anna Truelove (401)'], positions left: 3, available las: ['Halie Chmura (401)', 'Akash Krishna (210)']
name: Amy Clark (401), hours left: 0, shifts: ['Monday 3-4', 'Monday 1-2', 'Monday 2-3', 'Monday 10-11', 'Tues 1-2', 'Wed 10-11', 'Wed 1-2', 'Thur 1-2']
Shift: Thur 1-2, LAs assigned: ['Sarah Bost (210)', 'Nidhi Murlidhar (401)', 'James Bury (401)', 'Amy Clark (401)'], positions left: 2, available las: ['Sarah Bost (210)', 'Dana Rubin (401)', 'Nidhi Murlidhar (401)', 'James Bury (401)', 'Caleb Kang (401)']
name: Shannon Goad (401), hours left: 1, shifts: ['Wed 3-4', 'Wed 4-5', 'Monday 2-3', 'Wed 2-3']
Shift: Tues 1-2, LAs assigned: ['James Bury (401)', 'Amy Clark (401)', 'Angel Karafas (210)'], positions left: 3, available las: ['Sarah Bost (210)', 'Shannon Goad (401)', 'Dana R

True

### Create HTML file

In [57]:
full_days = {
    "Monday": "Monday",
    "Tues": "Tuesday",
    "Wed": "Wednesday",
    "Thur": "Thursday",
}

html = """<!doctype html>
<html lang="en">

<head>
    <!-- Required meta tags -->
    <meta charset="utf-8">

    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css"
        integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">

    <title>Home</title>
</head>

<body>

    <div class="container" style="margin-top:30px">
    
        <a href="altView.html">View shifts by LA</a>
        
        <br>

        <h1>Monday:</h1>
        <div class="row">"""
        

cols = 0
day_index = 0

for s in shifts:
        
    if days[day_index] not in s.title:
        day_index += 1
        html += "</div>\n"
        html += "<br><h1>" + full_days[days[day_index]] + ":</h1>"
        html += "<div class=\"row\">"
        cols = 0
        
    time = s.title.split(" ")[1]
    hour = time.split("-")[0]
        
    html += """<div class="col-3">
                   <div class="card border-dark mb-3" style="max-width: 18rem;">
                       <div class="card-header">""" + str(time + " am" if int(hour) == 10 else time + " pm") + """</div>
                       <div class="card-body text-dark">
                           <ul>"""
    
    for l in sorted(s.workers, key=lambda x: (x.name)):
        html += "<li><p class=\"card-text\">" + l.name + "</p></li>"

    html += """             </ul>
                       </div>
                   </div>
               </div>"""
    cols += 1
            
html +="""</div>
    </div>

    <!-- Optional JavaScript -->
    <!-- jQuery first, then Popper.js, then Bootstrap JS -->
    <script src="https://code.jquery.com/jquery-3.4.1.slim.min.js"
        integrity="sha384-J6qa4849blE2+poT4WnyKhv5vZF5SrPo0iEjwBvKU7imGFAV0wwj1yYfoRSJoZ+n"
        crossorigin="anonymous"></script>
    <script src="https://cdn.jsdelivr.net/npm/popper.js@1.16.0/dist/umd/popper.min.js"
        integrity="sha384-Q6E9RHvbIyZFJoft+2mJbHaEWldlvI9IOYy5n3zV9zzTtmI3UksdQRVvoxMfooAo"
        crossorigin="anonymous"></script>
    <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/js/bootstrap.min.js"
        integrity="sha384-wfSDF2E50Y2D1uUdj0O3uMBJnjuUD4Ih7YwaYd1iqfktj0Uod8GCExl3Og8ifwB6"
        crossorigin="anonymous"></script>
</body>

</html>"""

file = open("index.html", "w")
file.write(html)

15703

In [61]:
html = """<!doctype html>
<html lang="en">

<head>
    <!-- Required meta tags -->
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">

    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css"
        integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">

    <title>LA View</title>
</head>

<body>

    <div class="container" style="margin-top:30px">
    
        <a href="index.html">Home</a>
        <br>
        <br>"""
        
for l in sorted(LAs, key=lambda x: (x.name)):
    html += "<h3>" + l.name + "</h3>"
    html += "<ul>"
    for t in sorted_times:
        for s in l.shifts:
            if s.title == t:
                time = s.title.split(" ")[1]
                hour = time.split("-")[0]
                html += "<li>" + full_days[s.title.split(" ")[0]] + ": " + str(time + " am" if int(hour) == 10 else time + " pm") + "</li>\n"
                html += """
                """
    html += "</ul>"
    
html += """</div>

    <!-- Optional JavaScript -->
    <!-- jQuery first, then Popper.js, then Bootstrap JS -->
    <script src="https://code.jquery.com/jquery-3.4.1.slim.min.js"
        integrity="sha384-J6qa4849blE2+poT4WnyKhv5vZF5SrPo0iEjwBvKU7imGFAV0wwj1yYfoRSJoZ+n"
        crossorigin="anonymous"></script>
    <script src="https://cdn.jsdelivr.net/npm/popper.js@1.16.0/dist/umd/popper.min.js"
        integrity="sha384-Q6E9RHvbIyZFJoft+2mJbHaEWldlvI9IOYy5n3zV9zzTtmI3UksdQRVvoxMfooAo"
        crossorigin="anonymous"></script>
    <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/js/bootstrap.min.js"
        integrity="sha384-wfSDF2E50Y2D1uUdj0O3uMBJnjuUD4Ih7YwaYd1iqfktj0Uod8GCExl3Og8ifwB6"
        crossorigin="anonymous"></script>
</body>

</html>"""

file = open("altView.html", "w")
file.write(html)

5673